# **Face Beautifier App (Virtual Makeup)**

In [ ]:
# with croping and selection img from device

import cv2
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# -------------------------------
# Step 1: Select image via dialog
# -------------------------------
Tk().withdraw()  # Hide Tkinter window
file_path = askopenfilename(
    title="Select an image",
    filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp *.tif *.tiff *.webp")]
)

if not file_path:
    exit()

# -------------------------------
# Step 2: Load image and resize
# -------------------------------
img = cv2.imread(file_path)
img = cv2.resize(img, (300, 400))
final = img.copy()

# -------------------------------
# Step 3: Global variables
# -------------------------------
x1 = y1 = x2 = y2 = None
drawing = False
crop_done = False
mode = None
mask = None
crop = []

# -------------------------------
# Step 4: Mouse callback
# -------------------------------
def mouse_crop(event, x, y, flags, param):
    global x1, y1, x2, y2, drawing, crop_done, crop

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x1, y1 = x, y

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp = final.copy()
        cv2.rectangle(temp, (x1, y1), (x, y), (255, 255, 255), 2)
        cv2.imshow("makeup", temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        x2, y2 = x, y
        crop_done = True

        temp = final.copy()
        cv2.rectangle(temp, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.imshow("makeup", temp)

        crop = [y1, y2, x1, x2]  # Save raw coords

# -------------------------------
# Step 5: Trackbar callback
# -------------------------------
def nothing(a):
    pass

# -------------------------------
# Step 6: Create window + trackbars
# -------------------------------
cv2.namedWindow("makeup", cv2.WINDOW_FREERATIO)
cv2.setMouseCallback("makeup", mouse_crop)

cv2.createTrackbar("hmin", "makeup", 0, 255, nothing)
cv2.createTrackbar("hmax", "makeup", 255, 255, nothing)
cv2.createTrackbar("smin", "makeup", 0, 255, nothing)
cv2.createTrackbar("smax", "makeup", 255, 255, nothing)
cv2.createTrackbar("vmin", "makeup", 0, 255, nothing)
cv2.createTrackbar("vmax", "makeup", 255, 255, nothing)

cv2.createTrackbar("Face Brightness", "makeup", 0, 100, nothing)
cv2.createTrackbar("Lips Darkness", "makeup", 0, 100, nothing)

# -------------------------------
# Step 7: Main Loop
# -------------------------------
while True:
    display = final.copy()

    if crop_done:
        y1, y2, x1, x2 = crop

        # FIX 1 — ensure valid coordinates
        y_start = min(y1, y2)
        y_end   = max(y1, y2)
        x_start = min(x1, x2)
        x_end   = max(x1, x2)

        crop_image = display[y_start:y_end, x_start:x_end]

        # FIX 2 — avoid empty crop crash
        if crop_image.size == 0:
            continue

        hsv = cv2.cvtColor(crop_image, cv2.COLOR_BGR2HSV)

        # Trackbar reads
        hm = cv2.getTrackbarPos("hmin", "makeup")
        hx = cv2.getTrackbarPos("hmax", "makeup")
        sm = cv2.getTrackbarPos("smin", "makeup")
        sx = cv2.getTrackbarPos("smax", "makeup")
        vm = cv2.getTrackbarPos("vmin", "makeup")
        vx = cv2.getTrackbarPos("vmax", "makeup")
        face_bri = cv2.getTrackbarPos("Face Brightness", "makeup")
        lips_dar = cv2.getTrackbarPos("Lips Darkness", "makeup")

        # Mask
        lower = np.array([hm, sm, vm])
        upper = np.array([hx, sx, vx])
        mask = cv2.inRange(hsv, lower, upper)

        # Morphology cleanup
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # Brightness/darkness
        preview = crop_image.astype(np.float32)
        if mode == "face":
            preview += face_bri
        elif mode == "lips":
            preview -= lips_dar

        preview = np.clip(preview, 0, 255).astype(np.uint8)

        # Combine mask & crop
        masked = cv2.bitwise_and(preview, preview, mask=mask)
        mask_inv = cv2.bitwise_not(mask)
        bg = cv2.bitwise_and(crop_image, crop_image, mask=mask_inv)
        final_crop = cv2.add(bg, masked)

        # FIX 3 — put crop back safely
        display[y_start:y_end, x_start:x_end] = final_crop

        cv2.imshow("Mask (Selected Area)", mask)
        cv2.imshow("Preview", display)

    else:
        cv2.imshow("makeup", display)

    # -------------------------------
    # Step 8: Key commands
    # -------------------------------
    key = cv2.waitKey(1) & 0xFF

    if key == ord('f'):
        mode = "face"

    elif key == ord('l'):
        mode = "lips"

    elif key == ord('a') and crop_done:
        final[y_start:y_end, x_start:x_end] = display[y_start:y_end, x_start:x_end]
        crop_done = False

    elif key == ord('s'):
        cv2.imwrite("makeup_result.jpg", final)

        # Create a temp image with "Saved" message
        temp_saved = final.copy()
        cv2.putText(temp_saved, "Saved!", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1,
                    (0, 255, 0), 2, cv2.LINE_AA)

        # Show message for 800ms
        cv2.imshow("makeup", temp_saved)
        cv2.waitKey(800)

        print("Saved. Press 'd' to display Before/After")


    elif key == ord('d'):
        beautified = cv2.imread("makeup_result.jpg")
        both = np.hstack((img, beautified))
        cv2.imshow("Before and After", both)
        print("Press 'b' to save this combined image")

    elif key == ord('b'):
        cv2.imwrite("before_after.jpg", both)

    elif key == ord('r'):
        crop_done = False
        crop = []
        mask = None

        cv2.setTrackbarPos("hmin", "makeup", 0)
        cv2.setTrackbarPos("hmax", "makeup", 255)
        cv2.setTrackbarPos("smin", "makeup", 0)
        cv2.setTrackbarPos("smax", "makeup", 255)
        cv2.setTrackbarPos("vmin", "makeup", 0)
        cv2.setTrackbarPos("vmax", "makeup", 255)
        cv2.setTrackbarPos("Face Brightness", "makeup", 0)
        cv2.setTrackbarPos("Lips Darkness", "makeup", 0)

        try:
            cv2.destroyWindow("Mask (Selected Area)")
            cv2.destroyWindow("Preview")
        except:
            pass

    elif key == ord('q'):
        break

# -------------------------------
# Step 9: Cleanup
# -------------------------------
cv2.destroyAllWindows()


Saved. Press 'd' to display Before/After
Press 'b' to save this combined image
